# Preprocessing the Used Cars Dataset

In this notebook, I initiate the data cleaning process by addressing placeholder characters represented as '--' within the dataset. Following this, I tackle the issue of missing values using a comprehensive approach that includes the removal of missing entries in certain columns, as well as the implementation of mean imputation, mode imputation, and multiple imputation techniques. Subsequently, I apply one-hot encoding to convert categorical variables into a numerical format that is suitable for modeling. This is followed by normalization of the numerical columns to ensure that the data is standardized across all relevant features. Finally, I perform feature selection using the SelectKBest method to identify the most informative variables that will enhance the predictive accuracy of the models.

Data Source: [US Used Cars dataset (3 million)](https://www.kaggle.com/datasets/ananaymital/us-used-cars-dataset)


In [208]:
import pandas as pd
import numpy as np
from collections import Counter
import ast
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

### Data Loading

In [135]:
data = pd.read_csv('/Users/gulsumasenacakir/Desktop/portfolio/UsedCarsML/used_cars_data.csv')

/var/folders/4k/9sv3rzqs4qj4yhs2xt5y1n240000gn/T/ipykernel_19339/3220119872.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/Users/gulsumasenacakir/Desktop/portfolio/UsedCarsML/used_cars_data.csv')


In [136]:
data.shape

(3000040, 66)

### Data Cleaning

In [137]:
data.head()

,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,JF1VA2M67G9829723,35.4 in,NaN,NaN,NaN,Sedan,NaN,Guaynabo,17.0,NaN,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,SALRR2RV0L2433391,37.6 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,SALCJ2FXXLH862327,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020


Some features have the occurrences of '--' and these occurrences are replaced with NaN values to standardize missing data representations.

In [138]:
df = data.replace('--', np.nan)

The percentage of missing/null values for each feature are calculated. 

In [139]:
null_percentage = df.isnull().mean()*100
null_percentage = null_percentage.sort_values(ascending=False)
print(null_percentage)

bed_height                 100.000000
vehicle_damage_category    100.000000
combine_fuel_economy       100.000000
is_certified               100.000000
bed                         99.347742
                              ...    
franchise_dealer             0.000000
dealer_zip                   0.000000
daysonmarket                 0.000000
city                         0.000000
year                         0.000000
Length: 66, dtype: float64


The number of unique values for each feature is calculated.

In [140]:
unique_counts = {col: df[col].nunique() for col in df.columns}
unique_counts_df = pd.DataFrame(list(unique_counts.items()), columns=['Variable', 'Unique_Count'])

print(unique_counts_df.sort_values(by='Unique_Count', ascending=False))

                   Variable  Unique_Count
0                       vin       3000000
38               listing_id       3000000
12              description       2519325
40         main_picture_url       2415855
41            major_options        279972
..                      ...           ...
33                is_oemcpo             1
9      combine_fuel_economy             0
60  vehicle_damage_category             0
30             is_certified             0
3                bed_height             0

[66 rows x 2 columns]


The columns/features that have more than 50% missing values and the highest numbers of unique values are removed.

In [141]:
df = df.drop(['vehicle_damage_category', 'combine_fuel_economy', 'is_certified', 'bed', 'bed_height', 'cabin',
'bed_length', 'is_oemcpo', 'is_cpo', 'owner_count', 'description', 'vin', 'main_picture_url', 'listing_id'], axis=1)

Given the high percentage of null values in the 'has_accidents' column and the difficulty in imputing them, the missing values have been removed.

In [142]:
print(null_percentage[null_percentage.index == 'has_accidents'].values)

[47.55253263]


In [143]:
df = df.dropna(subset=['has_accidents'])

After removing the features with the highest number of unique values, I proceed to eliminate duplicates from the dataset.

In [144]:
df = df.drop_duplicates()

The features 'transmission_display' and 'wheel_system_display' are similar with 'transmission' and 'wheel_system'. Therefore, these are removed.

In [145]:
df = df.drop(['transmission_display', 'wheel_system_display'], axis=1)

The features 'engine_cylinders' and 'engine_type' are the same and 'engine_cylinders' is removed.

In [146]:
df = df.drop('engine_cylinders', axis=1)

Since the definition of 'sp_id' is not known, it is removed. Moreover, the feature 'listing_color' is the dominant color of 'exterior_color'. That's why 'exterior_color' is removed. 

In [147]:
df = df.drop(['sp_id', 'exterior_color'], axis=1)

The feature 'major_options' contains key car features such as Bluetooth and backup cameras, making it a crucial variable. Therefore, rows with missing values in this feature are removed from the dataset.

In [148]:
df = df.dropna(subset=['major_options'])

The percentage of missing values for each feature left are recalculated.

In [149]:
new_percentage = df.isnull().mean()*100
new_percentage = new_percentage.sort_values(ascending=False)
print(new_percentage)

franchise_make          34.885174
city_fuel_economy       14.817178
highway_fuel_economy    14.817178
interior_color          14.561728
torque                  12.122195
power                   10.365084
back_legroom             6.051339
front_legroom            4.087203
fuel_tank_volume         3.492641
maximum_seating          3.475588
height                   3.475055
width                    3.474722
length                   3.474523
wheelbase                3.473324
horsepower               3.321452
engine_displacement      3.321452
wheel_system             2.529057
engine_type              2.203666
trim_name                2.181818
trimId                   2.162168
fuel_type                1.733664
seller_rating            1.658328
transmission             1.485142
mileage                  1.099335
body_type                0.063613
daysonmarket             0.000000
theft_title              0.000000
sp_name                  0.000000
savings_amount           0.000000
price         

There are some features which have unit labels ('in' for inches, 'gal' for gallons, 'seats') at the end of values. These labels are removed and the features are converted the modified strings into floats for numerical analysis.

In [150]:
df['back_legroom'] = df['back_legroom'].str.replace(r'\s*in$', '', regex=True).astype(float)
df['front_legroom'] = df['front_legroom'].str.replace(r'\s*in$', '', regex=True).astype(float)
df['width'] = df['width'].str.replace(r'\s*in$', '', regex=True).astype(float)
df['wheelbase'] = df['wheelbase'].str.replace(r'\s*in$', '', regex=True).astype(float)
df['height'] = df['height'].str.replace(r'\s*in$', '', regex=True).astype(float)
df['length'] = df['length'].str.replace(r'\s*in$', '', regex=True).astype(float)
df['fuel_tank_volume'] = df['fuel_tank_volume'].str.replace(r'\s*gal$', '', regex=True).astype(float)
df['maximum_seating'] = df['maximum_seating'].str.replace(r'\s*seats$', '', regex=True).astype(float)

In [151]:
df.head()

,back_legroom,body_type,city,city_fuel_economy,daysonmarket,dealer_zip,engine_displacement,engine_type,fleet,frame_damaged,...,sp_name,theft_title,torque,transmission,trimId,trim_name,wheel_system,wheelbase,width,year
2,35.4,Sedan,Guaynabo,17.0,1233,969,2500.0,H4,False,False,...,FIAT de San Juan,False,"290 lb-ft @ 4,000 RPM",M,t58994,Base,AWD,104.3,78.9,2016
5,37.1,SUV / Crossover,San Juan,NaN,242,922,2000.0,I4,False,False,...,Land Rover San Juan,False,"269 lb-ft @ 1,200 RPM",A,t85614,P250 R-Dynamic S AWD,AWD,113.1,84.4,2020
9,33.8,SUV / Crossover,San Juan,NaN,510,922,2000.0,I4,False,False,...,Land Rover San Juan,False,"295 lb-ft @ 1,600 RPM",A,t85531,P300 R-Dynamic SE AWD,AWD,105.6,82.7,2020
10,NaN,Coupe,Guaynabo,NaN,1252,969,1700.0,I4,False,False,...,FIAT de San Juan,False,"258 lb-ft @ 4,250 RPM",A,t56834,Launch Edition Coupe RWD,RWD,93.7,73.5,2015
12,35.1,Sedan,Guaynabo,22.0,1233,969,3000.0,I6,False,False,...,FIAT de San Juan,False,"330 lb-ft @ 1,380 RPM",A,t58802,340i xDrive Sedan AWD,AWD,110.6,80.0,2016


In [152]:
df = df.dropna(subset=['franchise_make'])

In [153]:
new_percentage = df.isnull().mean()*100
new_percentage = new_percentage.sort_values(ascending=False)
print(new_percentage)

city_fuel_economy       14.360391
highway_fuel_economy    14.360391
interior_color          13.986599
torque                  12.714951
power                   11.088742
back_legroom             5.031865
front_legroom            3.604828
fuel_tank_volume         3.066953
maximum_seating          3.063270
height                   3.063066
length                   3.062861
width                    3.062759
wheelbase                3.062350
horsepower               3.051302
engine_displacement      3.051302
wheel_system             2.486113
trim_name                2.211242
trimId                   2.199376
engine_type              1.720628
transmission             1.612296
fuel_type                1.367091
mileage                  1.363818
seller_rating            0.985116
body_type                0.067413
salvage                  0.000000
savings_amount           0.000000
franchise_make           0.000000
sp_name                  0.000000
theft_title              0.000000
city          

In [154]:
df['interior_color'].nunique()

15894

In the raw dataset, there are three color-related features: exterior_color, interior_color, and listing_color. I plan to remove the exterior_color feature, as the listing color represents the dominant exterior color group, making it redundant. Furthermore, the interior_color feature will also be removed due to its high cardinality, with 15,894 unique values, which would require significant preprocessing and feature engineering to be useful for analysis.

In [155]:
df = df.drop('interior_color', axis=1)

## Imputation

Before determining the approach for features with missing value percentages exceeding 10%, I first apply mean imputation to numerical columns and mode imputation to categorical columns with lower percentages of missing values.

In [156]:
numeric_columns = df.select_dtypes(include=['number']).columns
print('Numeric columns:', numeric_columns)
categorical_columns = df.select_dtypes(include=['object', 'category']).columns
print('Categorical columns:', categorical_columns)
bool_columns = df.select_dtypes(include=['bool']).columns
print('Boolen columns:', bool_columns)

Numeric columns: Index(['back_legroom', 'city_fuel_economy', 'daysonmarket',
       'engine_displacement', 'front_legroom', 'fuel_tank_volume', 'height',
       'highway_fuel_economy', 'horsepower', 'latitude', 'length', 'longitude',
       'maximum_seating', 'mileage', 'price', 'savings_amount',
       'seller_rating', 'wheelbase', 'width', 'year'],
      dtype='object')
Categorical columns: Index(['body_type', 'city', 'dealer_zip', 'engine_type', 'fleet',
       'frame_damaged', 'franchise_make', 'fuel_type', 'has_accidents',
       'isCab', 'listed_date', 'listing_color', 'major_options', 'make_name',
       'model_name', 'power', 'salvage', 'sp_name', 'theft_title', 'torque',
       'transmission', 'trimId', 'trim_name', 'wheel_system'],
      dtype='object')
Boolen columns: Index(['franchise_dealer', 'is_new'], dtype='object')


### Mean imputation

The missing values of numerical columns are filled by using the mean.

In [157]:
for col in ['back_legroom', 'seller_rating', 'mileage', 'fuel_tank_volume', 'front_legroom', 'maximum_seating', 'height', 'width', 'wheelbase', 'length', 'horsepower', 'engine_displacement']:
    df[col] = df[col].fillna(df[col].mean()) ## there is no seller id 

### Mod imputation

Before filling missing values of categorical columns, their unique values are calculated.

In [158]:
unique_values_cat = {col: df[col].nunique() for col in categorical_columns}

unique_values_cat = pd.DataFrame(list(unique_values_cat.items()), columns=['Variable', 'Unique_Count'])
print(unique_values_cat.sort_values(by = 'Unique_Count', ascending=False))

          Variable  Unique_Count
12   major_options        136597
21          trimId         28911
17         sp_name         12829
22       trim_name          6434
2       dealer_zip          5646
1             city          3257
15           power          1654
19          torque          1634
10     listed_date          1358
14      model_name           982
13       make_name            60
6   franchise_make            48
3      engine_type            34
11   listing_color            15
0        body_type             9
7        fuel_type             8
23    wheel_system             5
20    transmission             4
9            isCab             2
16         salvage             2
8    has_accidents             2
18     theft_title             2
5    frame_damaged             2
4            fleet             2


Since 'trim_name' and 'trimId' have large number unique values, they are removed.

In [159]:
df = df.drop(['trim_name', 'trimId'], axis=1)

The missing values of categorical columns are filled by using the mod.

In [160]:
for col in ['body_type', 'fuel_type', 'engine_type', 'wheel_system', 'transmission']:
    most_common = df[col].mode()[0]  # Mod 
    df[col] = df[col].fillna(most_common)

### Multiple imputation

In [161]:
new_percentage_df = df.isnull().mean()*100
print(new_percentage_df[new_percentage_df.values > 0]) # features with missing values

city_fuel_economy       14.360391
highway_fuel_economy    14.360391
power                   11.088742
torque                  12.714951
dtype: float64


In [162]:
missing_values_columns = df[['city_fuel_economy', 'highway_fuel_economy', 'power', 'torque']]
missing_values_columns.head(5)

,city_fuel_economy,highway_fuel_economy,power,torque
2,17.0,23.0,"305 hp @ 6,000 RPM","290 lb-ft @ 4,000 RPM"
5,NaN,NaN,"247 hp @ 5,500 RPM","269 lb-ft @ 1,200 RPM"
9,NaN,NaN,"296 hp @ 5,500 RPM","295 lb-ft @ 1,600 RPM"
10,NaN,NaN,"237 hp @ 6,000 RPM","258 lb-ft @ 4,250 RPM"
12,22.0,33.0,"320 hp @ 5,500 RPM","330 lb-ft @ 1,380 RPM"


The features 'city_fuel_economy' and 'highway_fuel_economy' are numerical columns. The features 'power' and 'torque' are of mixed type. For example, the feature 'power' contains values formatted like '395 hp @ 5,600 RPM'. To simplify analysis, I separate this column into two columns: one for horsepower and another for RPM. Moreover, the feature 'torque' contains values formatted like '258 lb-ft @ 1,500 RPM'. I separate this column into two columns: one for lb-ft and another for torque RPM.

In [163]:
df[['hp', 'RPM']] = df['power'].str.split('@', expand=True)
df['hp'] = df['hp'].str.extract('(\d+)')
df['RPM'] = df['RPM'].str.extract('(\d+)')

In [164]:
df[['hp', 'RPM']].dtypes

hp     object
RPM    object
dtype: object

In [165]:
df['hp'] = pd.to_numeric(df['hp'], errors='coerce')
df['RPM'] = pd.to_numeric(df['RPM'], errors='coerce')

In [166]:
df[['torque_value', 'torque_rpm']] = df['torque'].str.split(' @ ', expand=True)
df['torque_value'] = df['torque_value'].str.extract('(\d+)')
df['torque_rpm'] = df['torque_rpm'].str.extract('(\d+)')

In [167]:
df[['torque_value', 'torque_rpm']].dtypes

torque_value    object
torque_rpm      object
dtype: object

In [168]:
df['torque_value'] = pd.to_numeric(df['torque_value'], errors='coerce')
df['torque_rpm'] = pd.to_numeric(df['torque_rpm'], errors='coerce')

In [169]:
df[['horsepower', 'hp']].head(10)

,horsepower,hp
2,305.0,305.0
5,247.0,247.0
9,296.0,296.0
10,237.0,237.0
12,320.0,320.0
23,246.0,246.0
36,201.0,201.0
38,160.0,160.0
40,311.0,311.0
41,310.0,310.0


The features 'horsepower' and 'hp' are the same. Therefore, the feature 'hp' is removed as well as the features 'power' and 'torque'.

In [170]:
df = df.drop(['torque', 'hp', 'power'], axis=1) 

In [171]:
new_percentage_df = df.isnull().mean()*100
print(new_percentage_df[new_percentage_df.values > 0]) # features with missing values

city_fuel_economy       14.360391
highway_fuel_economy    14.360391
RPM                     11.088742
torque_value            12.714951
torque_rpm              12.714951
dtype: float64


In [172]:
imputer = IterativeImputer()
columns_to_multiple_impute = ['city_fuel_economy', 'highway_fuel_economy', 'RPM', 'torque_value', 'torque_rpm']
imputed_data = imputer.fit_transform(df[columns_to_multiple_impute])
df[columns_to_multiple_impute] = imputed_data

/Users/gulsumasenacakir/anaconda3/lib/python3.11/site-packages/sklearn/impute/_iterative.py:796: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


The only feature with missing values is interior_color

In [173]:
new_percentage_df = df.isnull().mean()*100
print(new_percentage_df[new_percentage_df.values > 0])

Series([], dtype: float64)


## Columns with Boolean Values

Boolean values in specified columns are converted into integers (0 or 1)

In [174]:
for col in bool_columns:
    df[col] = df[col].astype(int)

## One-hot encoding

To manage the dimensionality of the dataset when applying one-hot encoding, I first identify the most frequent categories for each categorical variable and label all remaining categories as 'Other'. This reduction of categories simplifies the dataset, ensuring that only the most impactful categories are included. 

In [175]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns

top_categories_dfs = []

for col in categorical_columns:
    top_categories = df[col].value_counts(normalize=True).nlargest(10).reset_index() 
    top_categories.columns = ['Category', 'Percentage'] 
    top_categories['Percentage'] *= 100  
    top_categories['Variable'] = col 
    top_categories_dfs.append(top_categories) 

final_df = pd.concat(top_categories_dfs, ignore_index=True)

print(final_df)

            Category  Percentage      Variable
0    SUV / Crossover   46.688763     body_type
1              Sedan   27.490052     body_type
2       Pickup Truck   13.784563     body_type
3          Hatchback    2.879444     body_type
4            Minivan    2.868600     body_type
..               ...         ...           ...
133              FWD   45.182855  wheel_system
134              AWD   25.331185  wheel_system
135              4WD   19.055700  wheel_system
136              RWD    6.612245  wheel_system
137              4X2    3.818014  wheel_system

[138 rows x 3 columns]


The threshold for selecting the most frequent categories is set at 20%. Any category that makes up at least 20% of a variable will be retained, and categories below this threshold will be labeled as 'Other'.

In [176]:
## body_type
print(final_df.loc[final_df['Variable'] == 'body_type'])
top_categories = df['body_type'].value_counts().nlargest(2).index ## the number of the most frequent categories is 2
df['body_type'] = df['body_type'].apply(lambda x: x if x in top_categories else 'Other')
print('Unique Values: ', df['body_type'].unique())

          Category  Percentage   Variable
0  SUV / Crossover   46.688763  body_type
1            Sedan   27.490052  body_type
2     Pickup Truck   13.784563  body_type
3        Hatchback    2.879444  body_type
4          Minivan    2.868600  body_type
5            Coupe    2.491228  body_type
6            Wagon    1.932996  body_type
7      Convertible    0.957496  body_type
8              Van    0.906859  body_type
Unique Values:  ['Sedan' 'SUV / Crossover' 'Other']


In [177]:
## city
print(final_df.loc[final_df['Variable'] == 'city'])
df = df.drop('city', axis=1) ## since there is no frequent variable

        Category  Percentage Variable
9        Houston    1.054882     city
10   San Antonio    0.694798     city
11      Columbus    0.676589     city
12  Jacksonville    0.581658     city
13      Columbia    0.493683     city
14         Tampa    0.470462     city
15     Las Vegas    0.453788     city
16    Cincinnati    0.446831     city
17         Miami    0.416654     city
18  Indianapolis    0.413994     city


In [178]:
## dealer_zip
print(final_df.loc[final_df['Variable'] == 'dealer_zip'])
df = df.drop('dealer_zip', axis=1) ## since there is no frequent category

   Category  Percentage    Variable
19    76051    0.203775  dealer_zip
20    77074    0.187203  dealer_zip
21    33612    0.182395  dealer_zip
22    43228    0.180451  dealer_zip
23    45249    0.177996  dealer_zip
24    32225    0.176871  dealer_zip
25    30096    0.160912  dealer_zip
26    63011    0.160708  dealer_zip
27    46240    0.159276  dealer_zip
28    91401    0.150581  dealer_zip


In [179]:
## engine_type
print(final_df.loc[final_df['Variable'] == 'engine_type'])
top_categories = df['engine_type'].value_counts().nlargest(2).index  ## the number of the most frequent categories is 2
df['engine_type'] = df['engine_type'].apply(lambda x: x if x in top_categories else 'Other')
print('Unique Values: ', df['engine_type'].unique())

                Category  Percentage     Variable
29                    I4   49.210577  engine_type
30                    V6   27.447803  engine_type
31                    V8    9.193699  engine_type
32  V8 Flex Fuel Vehicle    2.953506  engine_type
33  V6 Flex Fuel Vehicle    2.590763  engine_type
34                    H4    2.424633  engine_type
35             I4 Hybrid    1.686870  engine_type
36                    I6    0.956268  engine_type
37                    I3    0.755869  engine_type
38          V8 Biodiesel    0.591479  engine_type
Unique Values:  ['Other' 'I4' 'V6']


In [180]:
## fuel_type
print(final_df.loc[final_df['Variable'] == 'fuel_type'])
top_categories = df['fuel_type'].value_counts().nlargest(1).index  ## the number of the most frequent categories is 1
df['fuel_type'] = df['fuel_type'].apply(lambda x: x if x in top_categories else 'Other')
print('Unique Values: ', df['fuel_type'].unique())

                  Category  Percentage   Variable
53                Gasoline   90.149967  fuel_type
54       Flex Fuel Vehicle    6.074983  fuel_type
55                  Hybrid    1.801954  fuel_type
56                  Diesel    0.955654  fuel_type
57               Biodiesel    0.681295  fuel_type
58                Electric    0.334407  fuel_type
59  Compressed Natural Gas    0.001432  fuel_type
60                 Propane    0.000307  fuel_type
Unique Values:  ['Gasoline' 'Other']


In [181]:
## listed_date
print(final_df.loc[final_df['Variable'] == 'listed_date'])
df = df.drop('listed_date', axis=1) ## since there is no frequent category

      Category  Percentage     Variable
65  2020-09-02    3.388778  listed_date
66  2020-09-03    2.870748  listed_date
67  2020-09-05    2.591172  listed_date
68  2020-09-04    2.435476  listed_date
69  2020-08-29    2.408675  listed_date
70  2020-08-28    2.342796  listed_date
71  2020-08-26    2.305662  listed_date
72  2020-08-27    2.292057  listed_date
73  2020-08-30    2.227610  listed_date
74  2020-09-06    2.159173  listed_date


In [182]:
## listing_color
print(final_df.loc[final_df['Variable'] == 'listing_color'])
top_categories = df['listing_color'].value_counts().nlargest(2).index  ## the number of the most frequent categories is 2
df['listing_color'] = df['listing_color'].apply(lambda x: x if x in top_categories else 'Other')
print('Unique Values: ', df['listing_color'].unique())

   Category  Percentage       Variable
75    WHITE   21.690451  listing_color
76    BLACK   19.788246  listing_color
77   SILVER   13.954785  listing_color
78     GRAY   12.612654  listing_color
79  UNKNOWN   12.490717  listing_color
80      RED    8.891924  listing_color
81     BLUE    8.235896  listing_color
82    BROWN    0.675566  listing_color
83    GREEN    0.651220  listing_color
84   ORANGE    0.347911  listing_color
Unique Values:  ['Other' 'WHITE' 'BLACK']


In [183]:
## make_name
print(final_df.loc[final_df['Variable'] == 'make_name'])
df = df.drop('make_name', axis=1) ## since there is no frequent category

      Category  Percentage   Variable
95        Ford   16.179837  make_name
96   Chevrolet   12.433840  make_name
97      Toyota    8.933661  make_name
98      Nissan    7.308475  make_name
99       Honda    6.149251  make_name
100       Jeep    5.473377  make_name
101    Hyundai    3.757966  make_name
102        GMC    3.671526  make_name
103      Dodge    3.410158  make_name
104        Kia    3.362692  make_name


In [184]:
## model_name
print(final_df.loc[final_df['Variable'] == 'model_name'])
df = df.drop('model_name', axis=1) ## since there is no frequent category

           Category  Percentage    Variable
105           F-150    3.873050  model_name
106  Silverado 1500    2.550049  model_name
107          Escape    2.388625  model_name
108         Equinox    2.108946  model_name
109        Explorer    1.929518  model_name
110            1500    1.908956  model_name
111           Rogue    1.755818  model_name
112           Camry    1.649327  model_name
113  Grand Cherokee    1.595724  model_name
114          Fusion    1.508874  model_name


In [185]:
## sp_name
print(final_df.loc[final_df['Variable'] == 'sp_name'])
df = df.drop('sp_name', axis=1) ## since there is no frequent category

                              Category  Percentage Variable
117                Honda of Fort Myers    0.077848  sp_name
118                      ALM Kia South    0.073040  sp_name
119                Elder Ford of Tampa    0.073040  sp_name
120                        Jordan Ford    0.069562  sp_name
121                       Preston Ford    0.066902  sp_name
122  Bill Luke Chrysler Jeep Dodge Ram    0.065777  sp_name
123                     MajorWorld.com    0.061992  sp_name
124           Shea Chevrolet Buick GMC    0.061071  sp_name
125                  Willis Automotive    0.060355  sp_name
126                     Sanderson Ford    0.059537  sp_name


In [186]:
## transmission
print(final_df.loc[final_df['Variable'] == 'transmission'])
top_categories = df['transmission'].value_counts().nlargest(1).index ## the number of the most frequent categories is 1
df['transmission'] = df['transmission'].apply(lambda x: x if x in top_categories else 'Other')
print('Unique Values: ', df['transmission'].unique())

        Category  Percentage      Variable
129            A   83.386630  transmission
130          CVT   14.420746  transmission
131            M    1.739962  transmission
132  Dual Clutch    0.452662  transmission
Unique Values:  ['Other' 'A']


In [187]:
## wheel_system
print(final_df.loc[final_df['Variable'] == 'wheel_system'])
top_categories = df['wheel_system'].value_counts().nlargest(2).index ## the number of the most frequent categories is 1
df['wheel_system'] = df['wheel_system'].apply(lambda x: x if x in top_categories else 'Other')
print('Unique Values: ', df['wheel_system'].unique())

    Category  Percentage      Variable
133      FWD   45.182855  wheel_system
134      AWD   25.331185  wheel_system
135      4WD   19.055700  wheel_system
136      RWD    6.612245  wheel_system
137      4X2    3.818014  wheel_system
Unique Values:  ['AWD' 'Other' 'FWD']


In [188]:
## franchise_make
print(final_df.loc[final_df['Variable'] == 'franchise_make'])
df = df.drop('franchise_make', axis=1)

     Category  Percentage        Variable
43       Ford   16.291954  franchise_make
44  Chevrolet   15.036673  franchise_make
45       Jeep    8.531738  franchise_make
46     Toyota    8.475986  franchise_make
47      Honda    6.497877  franchise_make
48     Nissan    5.418546  franchise_make
49      Buick    3.577413  franchise_make
50        Kia    3.464171  franchise_make
51    Hyundai    3.276252  franchise_make
52     Subaru    2.831876  franchise_make


'major_options' consists of a list of specific features, such as 'Steel Wheels', 'Bluetooth', 'Backup Camera'. Initially, the five most frequent features are identified, and new columns are created for each of these top features.

In [189]:
options_counter = Counter()

for options_string in df['major_options']:
   
    options_list = ast.literal_eval(options_string)
    
    options_counter.update(options_list)

options_count_df = pd.DataFrame(list(options_counter.items()), columns=['Option', 'Count'])

total_options_count = sum(options_counter.values())

options_count_df['Percentage'] = options_count_df['Count'].apply(lambda x: (x / total_options_count) * 100)

options_count_df.sort_values(by='Percentage', ascending=False) ## there are 144 major options

,Option,Count,Percentage
2,Backup Camera,815432,12.871205
1,Bluetooth,769155,12.140744
0,Alloy Wheels,722379,11.402407
3,Heated Seats,480406,7.582979
6,Navigation System,417319,6.587181
...,...,...,...
138,Preferred Premium Accessory Package,7,0.000110
139,Luxury Sound Insulation Package,5,0.000079
134,Appearance and Protection Package,3,0.000047
141,Grand Tour Package,3,0.000047


In [190]:
df['Bluetooth'] = df['major_options'].apply(lambda options: 1 if 'Bluetooth' in options else 0)
df['Backup_Camera'] = df['major_options'].apply(lambda options: 1 if 'Backup Camera' in options else 0)
df['Alloy_Wheels'] = df['major_options'].apply(lambda options: 1 if 'Alloy Wheels' in options else 0)
df['Heated_Seats'] = df['major_options'].apply(lambda options: 1 if 'Heated Seats' in options else 0)
df['Navigation_System'] = df['major_options'].apply(lambda options: 1 if 'Navigation System' in options else 0)

In [191]:
df = df.drop('major_options', axis=1) ## removed

In [192]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns
print('Categorical columns:', categorical_columns)
unique_values = {col: df[col].nunique() for col in categorical_columns}

unique_values = pd.DataFrame(list(unique_values.items()), columns=['Variable', 'Unique_Count'])
print(unique_values.sort_values(by = 'Unique_Count', ascending=False))

Categorical columns: Index(['body_type', 'engine_type', 'fleet', 'frame_damaged', 'fuel_type',
       'has_accidents', 'isCab', 'listing_color', 'salvage', 'theft_title',
       'transmission', 'wheel_system'],
      dtype='object')
         Variable  Unique_Count
0       body_type             3
1     engine_type             3
7   listing_color             3
11   wheel_system             3
2           fleet             2
3   frame_damaged             2
4       fuel_type             2
5   has_accidents             2
6           isCab             2
8         salvage             2
9     theft_title             2
10   transmission             2


In [193]:
df[categorical_columns].head(5)

,body_type,engine_type,fleet,frame_damaged,fuel_type,has_accidents,isCab,listing_color,salvage,theft_title,transmission,wheel_system
2,Sedan,Other,False,False,Gasoline,False,False,Other,False,False,Other,AWD
5,SUV / Crossover,I4,False,False,Gasoline,False,False,Other,False,False,A,AWD
9,SUV / Crossover,I4,False,False,Gasoline,False,False,WHITE,False,False,A,AWD
10,Other,I4,False,False,Gasoline,False,False,Other,False,False,A,Other
12,Sedan,Other,False,False,Gasoline,False,False,Other,False,False,A,AWD


The features 'fuel_type' ve 'transmission' have 2 different values. If the value is other, set 0 otherwise 1

In [194]:
df['fuel_type'] = df['fuel_type'].map({'Other': 0, 'Gasoline': 1})
df['transmission'] = df['transmission'].map({'Other': 0, 'A': 1})

The features 'isCab', 'salvage', 'theft_title', 'has_accidents', 'frame_damaged' and 'fleet' contain true/false values and these columns are converted into 1/0.

In [195]:
columns_to_convert = ['isCab', 'salvage', 'theft_title', 'has_accidents', 'frame_damaged', 'fleet']

df[columns_to_convert] = df[columns_to_convert].astype(int)

There are still some categorical columns.

In [196]:
categorical_columns_last = df.select_dtypes(include=['object', 'category']).columns
print('Categorical columns:', categorical_columns_last)

Categorical columns: Index(['body_type', 'engine_type', 'listing_color', 'wheel_system'], dtype='object')


One-hot encoding is applied to the categorical columns in the dataset to convert categorical data into a numerical format.

In [197]:
df = pd.get_dummies(df, columns=categorical_columns_last)
df = df.astype(int)

## Normalization

First, the features and the target variable are defined.

In [230]:
X = df.drop('price', axis=1) # features
y = df['price'] # label or target variable


Before feature selection, the distributions of target variable and features are analyzed to check if needed normalization of the data set.

In [206]:
print(y.agg(['min', 'mean', 'max']))

min     2.490000e+02
mean    2.586880e+04
max     3.195000e+06
Name: price, dtype: float64


Based on the statistics of the target variable 'price', which shows a minimum value of 200, a mean of 23,226.65, and a maximum of 3,195,000, scaling is necessary to normalize the data distribution. This step will help reduce the influence of extreme values, ensuring that the machine learning models can operate more efficiently and effectively, without being skewed by the wide range of values in the target variable.

In [207]:
print(X.agg(['min', 'mean', 'max']))

      back_legroom  city_fuel_economy  daysonmarket  engine_displacement  \
min       0.000000         -12.000000      0.000000           700.000000   
mean     37.314842          21.455286     49.493916          2999.578203   
max      59.000000         127.000000   3573.000000          8400.000000   

         fleet  frame_damaged  franchise_dealer  front_legroom  \
min   0.000000       0.000000               1.0        0.00000   
mean  0.186689       0.002121               1.0       41.92513   
max   1.000000       1.000000               1.0       67.00000   

      fuel_tank_volume  fuel_type  ...  body_type_Sedan  engine_type_I4  \
min           1.000000     0.0000  ...         0.000000        0.000000   
mean         18.340053     0.9015  ...         0.274901        0.492106   
max          64.000000     1.0000  ...         1.000000        1.000000   

      engine_type_Other  engine_type_V6  listing_color_BLACK  \
min            0.000000        0.000000             0.000000   
m

In [228]:
unique_values = {col: df[col].nunique() for col in df.columns}

unique_values = pd.DataFrame(list(unique_values.items()), columns=['Variable', 'Unique_Count'])
print(unique_values[unique_values['Unique_Count'] > 2]['Variable'])

0             back_legroom
1        city_fuel_economy
2             daysonmarket
3      engine_displacement
7            front_legroom
8         fuel_tank_volume
11                  height
12    highway_fuel_economy
13              horsepower
16                latitude
17                  length
18               longitude
19         maximum_seating
20                 mileage
21                   price
23          savings_amount
24           seller_rating
27               wheelbase
28                   width
29                    year
30                     RPM
31            torque_value
32              torque_rpm
Name: Variable, dtype: object


Given the substantial range in the statistics of the features, such as 'daysonmarket' which varies from 0 to 3573, and 'engine_displacement' ranging from 1000 to 8400, normalization is essential. Normalizing these features will scale their values to a common range, typically [0, 1], making it easier for machine learning algorithms to process the data without bias towards larger scale features. 

In [229]:
scaler = MinMaxScaler()
norm_columns = ['back_legroom', 'city_fuel_economy', 'daysonmarket', 'engine_displacement', 'front_legroom', 'fuel_tank_volume', 'height', 'highway_fuel_economy',
                'horsepower', 'latitude', 'length', 'longitude', 'maximum_seating', 'mileage', 'savings_amount', 'seller_rating', 'wheelbase', 'width', 'year', 'RPM',
                'torque_value', 'torque_rpm', 'price']
df[norm_columns] = scaler.fit_transform(df[norm_columns])

In [232]:
X = df.drop('price', axis=1) # features
y = df['price'] # label or target variable

## Feature Selection

There are 49 features and I need to perform dimensionality reduction/feature selection to increase model training efficiency and potentially enhance model accuracy by eliminating irrelevant features that do not contribute significantly to the predictive power of the model. This process will also help in simplifying the model, making it faster to train and easier to interpret. For the feature selection process, the SelectKBest method is employed.

In [233]:
selector = SelectKBest(score_func=f_regression, k=10)

X_new = selector.fit_transform(X, y)

selected_features = X.columns[selector.get_support()]
selected_scores = selector.scores_[selector.get_support()]

# create dataframe for selected features
features_scores_df = pd.DataFrame({'Feature': selected_features, 'Score': selected_scores})

print(features_scores_df.sort_values(by="Score", ascending=False))

               Feature          Score
2           horsepower  499048.063586
8         torque_value  171765.763974
1     fuel_tank_volume  166502.068791
4              mileage  161524.595421
9     wheel_system_FWD  154705.234057
0  engine_displacement  151355.017534
6                width  145109.060237
7                 year  141083.235022
5            wheelbase  126213.501355
3               length  119934.685950


The scaled feature set and the scaled target variable are horizontally concatenated for modeling.

In [234]:
X_filtered = X[selected_features]
y_scaled_series = pd.Series(y, name='price')
df_filtered = pd.concat([X_filtered, y_scaled_series], axis=1)

Save the cleaned and preprocessed dataset to a CSV file.

In [235]:
df_filtered.to_csv('filtered_used_cars_data.csv', index=False)